In [30]:
# --- ユーザー設定セクション ---
# 1. 解析対象の製品名と不具合事象 (中心テーマ)
PRODUCT_NAME = "自動車"
FAILURE_EVENT = "エンジンがかからない" # 連関図の中心テーマ

# 2. Google Gemini APIキー (スクリプト内直接指定の場合。非推奨)
# .envファイルや環境変数で設定する場合は、この値は空のままで問題ありません。
GEMINI_API_KEY_IN_SCRIPT = ""  # 例: "AIzaSy*******************"

# 3. LLMモデル設定
# 利用可能なモデル例: "gemini-pro", "gemini-1.0-pro", "gemini-1.5-flash-latest", "gemini-1.5-pro-latest"
GEMINI_MODEL_NAME = "gemini-2.0-flash"
TEMPERATURE_SETTING = 0.7  # 連関図の要因展開にある程度の多様性を許容するため少し高めに設定

# 4. PlantUML出力設定
OUTPUT_FILENAME_PREFIX_RELATION = "RelationshipDiagram" # 出力ファイル名の接頭辞
# PlantUMLで使用するデフォルトフォント。日本語を表示する場合、
# PlantUMLレンダリング環境（VSCode拡張機能が利用するJava/Graphviz環境など）に
# インストールされている日本語フォントを指定してください。
DEFAULT_FONT_NAME = "Yu Gothic" # 例: "MS Gothic", "IPAexGothic", "Noto Sans CJK JP"

# --- ユーザー設定セクションここまで ---

print("ユーザー設定が読み込まれました。")
print(f"  製品名: {PRODUCT_NAME}")
print(f"  不具合事象: {FAILURE_EVENT}")
print(f"  使用LLMモデル: {GEMINI_MODEL_NAME}")

ユーザー設定が読み込まれました。
  製品名: 自動車
  不具合事象: エンジンがかからない
  使用LLMモデル: gemini-2.0-flash


In [31]:
# --- 必要なライブラリのインポートとAPIキーの読み込み ---
import os
import time
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv

# .envファイルから環境変数を読み込む (存在する場合)
# ノートブックと同じディレクトリに .env ファイルを置くと自動で読み込まれます。
load_dotenv()

# APIキーの取得 (環境変数 -> .env -> スクリプト内直接指定 の優先順位)
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY") or GEMINI_API_KEY_IN_SCRIPT

if not GEMINI_API_KEY:
    print("警告: Gemini APIキーが設定されていません。LLMの呼び出しに失敗する可能性があります。")
    print("      スクリプト冒頭の指示に従い、APIキーを設定してください。")
else:
    print("Gemini APIキーが読み込まれました。")

Gemini APIキーが読み込まれました。


In [32]:
# --- LLMによる連関図PlantUMLデータ生成関数 ---
def generate_relationship_diagram_puml(product_name: str, central_theme: str, llm_instance, font_name: str, retries=3) -> str | None:
    """
    製品名と中心テーマを基に、LLMを使用して連関図のPlantUML記述を生成する。
    """
    plantuml_example_structure = f"""@startuml
left to right direction
skinparam defaultFontName "{font_name}"

' 中心テーマ (最も重要な問題) - {product_name}の{central_theme}
rectangle "{central_theme} ({product_name})" as T0 #LightCoral;line:Firebrick;line.bold

' 主要因 (一次要因) - T0から派生するものを具体的に記述させる
rectangle "（{central_theme}の主要因1の具体的な名称）" as F1
rectangle "（{central_theme}の主要因2の具体的な名称）" as F2
' rectangle "（主要因3...）" as F3 ...

' 二次要因 - F1, F2などから派生するものを具体的に記述させる
rectangle "（主要因1から派生する具体的要因Aの名称）" as S1_A
rectangle "（主要因1から派生する具体的要因Bの名称）" as S1_B
rectangle "（主要因2から派生する具体的要因Cの名称）" as S2_C
' rectangle "（二次要因...）" as S_...

' 関係性の定義 (LLMに適切に設定させる)
T0 --> F1
T0 --> F2
F1 --> S1_A
F1 --> S1_B
F2 --> S2_C

@enduml
"""

    prompt_template = f"""
あなたはシステム思考と問題解決の専門家です。以下の製品と不具合事象（中心テーマ）について、それらの原因や関連する事象を明らかにするための「連関図」をPlantUML形式で記述してください。

製品名: {product_name}
中心テーマ（不具合事象）: {central_theme}

作成手順と指示：
1.  中心テーマ「{central_theme} ({product_name})」を、ID `T0`、色 `#LightCoral` などで最も重要な要素として定義してください。
2.  この中心テーマに直接関連する主要な要因（一次要因）を、具体的な名称で3～5個程度洗い出してください。これらはID `F1`, `F2`... とし、色 `#Orange` などで表現してください。
3.  それぞれの一次要因からさらに派生する具体的な要因や事象（二次要因）を、各一次要因ごとに具体的な名称で2～3個程度洗い出してください。これらはID `S1_A`, `S1_B`, `S2_A`... のように命名規則を工夫し（例: S<一次要因IDの数字>_<連番アルファベット>）、色 `#Gold` や `White` などで表現してください。
4.  各要素間の関係性を矢印（`-->` または `->`）で示してください。
5.  要素のIDは、例（`T0`, `F1`, `S1_A` など）を参考に、ユニークかつ階層や関連が推測しやすいように設定してください。
6.  出力は、必ず `@startuml` で始まり `@enduml` で終わる完全なPlantUMLコードのみとしてください。他の説明文は一切含めないでください。
7.  以下のPlantUMLの基本書式とスタイル指定を必ず含めてください。
    - `left to right direction`
    - `skinparam defaultFontName "{font_name}"`
8.  洗い出す要因の数は、図が過度に複雑にならない範囲で、問題の構造を捉えるのに十分な程度としてください。要素名は日本語で具体的に記述してください。

以下のPlantUMLの記述例の構造を参考にし、「（...）で示されたプレースホルダー部分」や要素名を、あなたが分析した具体的な内容に置き換えて、完全なPlantUMLコードを生成してください。
```plantuml
{plantuml_example_structure}
"""

    for attempt in range(retries):
        try:
            print(f"\nLLM ({llm_instance.model}) に連関図のPlantUML記述を問い合わせています... (試行 {attempt + 1}/{retries})")
            response = llm_instance.invoke(prompt_template)
            
            content = response.content if hasattr(response, 'content') else str(response)
            
            content_stripped = content.strip()
            # Markdownのコードブロック形式を除去
            if content_stripped.startswith("```plantuml"):
                content_stripped = content_stripped[10:] 
                if content_stripped.endswith("```"):
                    content_stripped = content_stripped[:-3]
            elif content_stripped.startswith("```puml"):
                content_stripped = content_stripped[7:]
                if content_stripped.endswith("```"):
                    content_stripped = content_stripped[:-3]
            elif content_stripped.startswith("```"): # その他の ``` block
                content_stripped = content_stripped[3:]
                if content_stripped.endswith("```"):
                    content_stripped = content_stripped[:-3]
            
            puml_code = content_stripped.strip()
            
                        # ===== 追加ここから =====
            content = response.content if hasattr(response, 'content') else str(response)

            print(f"\n--- Debug: Attempt {attempt + 1} ---")
            print("Raw content from LLM (START):")
            print(content) # 完全な内容を確認
            print("Raw content from LLM (END)")
            # ===== 追加ここまで =====

            # if puml_code.startswith("@startuml") and puml_code.endswith("@enduml"):
            #     print("PlantUMLコードの生成に成功したようです。")
            #     return puml_code
            # else:
            #     print(f"エラー: LLMの応答が期待されるPlantUML形式（@startuml...@enduml）ではありません (試行 {attempt + 1})。")
            #     if attempt == retries - 1: return None

        except Exception as e:
            print(f"LLM呼び出しまたは処理中に予期せぬエラーが発生しました (試行 {attempt + 1}): {e}")
            if attempt == retries - 1: return None
        
        # if attempt < retries - 1:
        #     wait_time = 10 * (attempt + 1)
        #     print(f"{wait_time}秒待機して再試行します...")
        #     time.sleep(wait_time)
            
    return puml_code

In [33]:
# --- PlantUMLファイル保存関数 ---
def save_puml_file(puml_content: str, filename_base: str) -> str | None:
    """
    PlantUMLの内容をファイルに保存する。
    """
    if not puml_content:
        print("エラー: 保存するPlantUMLの内容がありません。")
        return None
    
    try:
        puml_filename = f"{filename_base}.puml" # 拡張子を .puml に
        with open(puml_filename, 'w', encoding='utf-8') as f:
            f.write(puml_content)
        return puml_filename
    except Exception as e:
        print(f"PlantUMLファイルの保存中にエラーが発生しました: {e}")
        return None

print("関数 'save_puml_file' が定義されました。")

関数 'save_puml_file' が定義されました。


In [34]:
# 1. 設定値の再検証 (APIキーは特に重要)
if not PRODUCT_NAME or not FAILURE_EVENT:
    print("エラー: 「ユーザー設定セクション」で 'PRODUCT_NAME' と 'FAILURE_EVENT' を設定してください。")
elif not GEMINI_API_KEY:
    print("エラー: Gemini APIキーが設定されていません。")
    print("       「APIキーの設定について」セクションの指示に従い、APIキーを設定してください。")
else:
    print(f"\n--- 設定情報再確認 (連関図) ---")
    print(f"製品名: {PRODUCT_NAME}")
    print(f"中心テーマ: {FAILURE_EVENT}")
    print(f"使用モデル: {GEMINI_MODEL_NAME}")
    print(f"PlantUMLフォント: {DEFAULT_FONT_NAME}")
    print(f"--------------------------------")

    # 2. LLMインスタンスの初期化
    llm_initialization_error = False
    try:
        llm = ChatGoogleGenerativeAI(
            model=GEMINI_MODEL_NAME,
            google_api_key=GEMINI_API_KEY,
            temperature=TEMPERATURE_SETTING,
        )
        print(f"LLM ({GEMINI_MODEL_NAME}) の準備ができました。")
    except Exception as e:
        print(f"LLMの初期化に失敗しました: {e}")
        llm_initialization_error = True

    if not llm_initialization_error:
        # 3. LLMによる連関図PlantUMLデータの生成
        print(f"\n製品「{PRODUCT_NAME}」の不具合「{FAILURE_EVENT}」について連関図のPlantUML記述を生成します。")
        plantuml_output_code = generate_relationship_diagram_puml(
            PRODUCT_NAME,
            FAILURE_EVENT,
            llm,
            DEFAULT_FONT_NAME
        )

        # 4. PlantUMLファイルの保存
        if plantuml_output_code:
            safe_product_name = "".join(c if c.isalnum() else "_" for c in PRODUCT_NAME)
            safe_failure_event = "".join(c if c.isalnum() else "_" for c in FAILURE_EVENT)
            output_filename_base = f"{OUTPUT_FILENAME_PREFIX_RELATION}_{safe_product_name}_{safe_failure_event}"
            
            saved_puml_file = save_puml_file(plantuml_output_code, output_filename_base)
            
            if saved_puml_file:
                print(f"\n生成されたPlantUMLファイル: {os.path.abspath(saved_puml_file)}")
                print("この .puml ファイルをPlantUMLに対応したツールで開くと、図として表示・エクスポートできます。")
            else:
                print("\nPlantUMLファイルの保存に失敗しました。")
        else:
            print("\n連関図のPlantUML記述の生成に失敗しました。")
            print("LLMの応答や設定、APIキー等を確認してください。")


--- 設定情報再確認 (連関図) ---
製品名: 自動車
中心テーマ: エンジンがかからない
使用モデル: gemini-2.0-flash
PlantUMLフォント: Yu Gothic
--------------------------------
LLM (gemini-2.0-flash) の準備ができました。

製品「自動車」の不具合「エンジンがかからない」について連関図のPlantUML記述を生成します。

LLM (models/gemini-2.0-flash) に連関図のPlantUML記述を問い合わせています... (試行 1/3)

--- Debug: Attempt 1 ---
Raw content from LLM (START):
```plantuml
@startuml
left to right direction
skinparam defaultFontName "Yu Gothic"

' 中心テーマ (最も重要な問題) - 自動車のエンジンがかからない
rectangle "エンジンがかからない (自動車)" as T0 #LightCoral;line:Firebrick;line.bold

' 主要因 (一次要因) - T0から派生するものを具体的に記述させる
rectangle "バッテリー上がり" as F1 #Orange
rectangle "燃料供給の問題" as F2 #Orange
rectangle "点火系の問題" as F3 #Orange
rectangle "スターターモーターの故障" as F4 #Orange

' 二次要因 - F1, F2などから派生するものを具体的に記述させる
rectangle "ライト消し忘れ" as S1_A #Gold
rectangle "バッテリーの寿命" as S1_B #Gold
rectangle "燃料ポンプの故障" as S2_A #Gold
rectangle "燃料フィルターの詰まり" as S2_B #Gold
rectangle "スパークプラグの不良" as S3_A #Gold
rectangle "イグニッションコイルの故障" as S3_B #Gold
rectangle "ソレノイドスイッチの故障" as S4_A #